In [1]:
import torch
from transformer import Transformer
from encoder import Encoder
from decoder import Decoder
from config import *

# 设备设置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 构建 Encoder 和 Decoder（与训练时结构一致）
enc = Encoder(INPUT_DIM, HID_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, MAX_LEN)
dec = Decoder(OUTPUT_DIM, HID_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, MAX_LEN)

# 构建 Transformer 模型
base_model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

# 加载训练好的权重
base_model.load_state_dict(torch.load("best_transformer.pt", map_location=device))
base_model.eval()  # 设置为评估模式

print("✅ 基础模型权重加载完成，可以用于评估或绘制对照曲线")


✅ 基础模型权重加载完成，可以用于评估或绘制对照曲线


In [ ]:
# translate.py — 兼容词表差异的推理脚本
import torch
from transformer import Transformer
from encoder import Encoder
from decoder import Decoder
from dataset_loader import get_dataloaders, basic_tokenizer
from config import *
from utils import make_src_mask, make_trg_mask
import sys

# ---------- 加载词表（只取词表，不重新训练） ----------
print("加载数据与词表...")
_, _, _, vocab_en, vocab_de = get_dataloaders()  # 只需要 vocab
print("EN vocab size:", len(vocab_en), "DE vocab size:", len(vocab_de))

# 打印 special tokens 检查（可注释掉）
print("Check DE special tokens present:",
      {k: (k in vocab_de) for k in ["<sos>", "<bos>", "<eos>", "<pad>", "<unk>"]})

# ---------- 安全取 token 索引的工具 ----------
def get_token_index(vocab, keys, default=None):
    """
    尝试按 keys 列表顺序取到第一个存在的索引；
    如果都不存在，则返回 default（或抛错）。
    """
    for k in keys:
        if k in vocab:
            return vocab[k], k
    return default, None

# ---------- 构建模型并加载权重 ----------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

enc = Encoder(INPUT_DIM, HID_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, MAX_LEN)
dec = Decoder(OUTPUT_DIM, HID_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, MAX_LEN)
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

state = torch.load("best_transformer.pt", map_location=device)
model.load_state_dict(state)
model.eval()
print("模型加载完成。")

# ---------- 准备 start/end/pad/unk 索引（兼容多种命名） ----------
sos_idx, sos_name = get_token_index(vocab_de, ["<sos>", "<bos>"], default=None)
eos_idx, eos_name = get_token_index(vocab_de, ["<eos>"], default=None)
pad_idx, pad_name = get_token_index(vocab_de, ["<pad>"], default=None)
unk_idx, unk_name = get_token_index(vocab_de, ["<unk>"], default=None)

# 如果连 eos 或 pad 都没有，那说明词表异常，要提醒用户
if eos_idx is None or pad_idx is None:
    print("ERROR: target vocab seems to miss essential tokens. Found keys:", list(vocab_de.keys())[:20])
    raise SystemExit("词表缺少 <eos> 或 <pad>，无法安全推理。")

print(f"Using target start token: {sos_name}, end token: {eos_name}, pad: {pad_name}, unk: {unk_name}")

# 反向词表（索引->token）
itos_de = {i: w for w, i in vocab_de.items()}

# ---------- 翻译函数（兼容命名、稳健） ----------
def translate_sentence(sentence, model, vocab_en, vocab_de, max_len=MAX_LEN, device=device):
    model.eval()

    # 1) 分词（与你训练时的简单 tokenizer 一致）
    tokens = basic_tokenizer(sentence)

    # 2) src 数值化（没有在这里添加 start/end —— 取决于训练时做法）
    src_indexes = [vocab_en.get(tok, vocab_en.get("<unk>", 0)) for tok in tokens]
    if len(src_indexes) == 0:
        src_indexes = [vocab_en.get("<unk>", 0)]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # 3) 构造 src_mask（使用你的 utils 中方法）
    try:
        src_mask = make_src_mask(src_tensor, vocab_en.get("<pad>", SRC_PAD_IDX)).to(device)
    except Exception:
        # 兼容：如果 make_src_mask 接口不同，用简单 mask
        src_mask = (src_tensor != vocab_en.get("<pad>", SRC_PAD_IDX)).unsqueeze(1).unsqueeze(2).to(device)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    # 4) 生成起始 token：如果没有 sos，就用 bos；如果都没有就用 pad（退化）
    start_idx = sos_idx if sos_idx is not None else pad_idx
    end_idx = eos_idx

    trg_indexes = [start_idx]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        # trg mask: 用提供的函数 if available
        try:
            trg_mask = make_trg_mask(trg_tensor, vocab_de.get("<pad>", TRG_PAD_IDX), device).to(device)
        except Exception:
            # 简单 causal mask fallback
            sz = trg_tensor.shape[1]
            trg_mask = torch.tril(torch.ones((1, 1, sz, sz), device=device)).bool()

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
            # output: [batch, trg_len, vocab_size]
            next_token = output.argmax(dim=-1)[:, -1].item()

        trg_indexes.append(next_token)

        if next_token == end_idx:
            break

    # 5) 把索引转换回 token（去掉起始符和结束符）
    words = []
    for idx in trg_indexes[1:]:  # 跳过 start token
        if idx == end_idx:
            break
        word = itos_de.get(idx, "<unk>")
        words.append(word)

    return " ".join(words)


# ---------- 交互示例 ----------
if __name__ == "__main__":
    print("\n输入英文，回车翻译（输入 quit 退出）")
    while True:
        s = input("> ").strip()
        if s.lower() in ["quit", "exit"]:
            break
        print("Translating ...")
        try:
            out = translate_sentence(s, model, vocab_en, vocab_de)
            print("德语：", out)
        except Exception as e:
            print("推理出错：", e)
            raise


加载数据与词表...
 数据加载完成: train=206112, dev=888, test=1568
🧩 分词中...


Tokenizing DE: 100%|██████████| 206112/206112 [00:00<00:00, 275284.01it/s]


 构建词典中...
EN vocab size: 59616, DE vocab size: 81512
 数值化中...
 DataLoader 创建完成: train=25764 batches
EN vocab size: 59616 DE vocab size: 81512
Check DE special tokens present: {'<sos>': False, '<bos>': True, '<eos>': True, '<pad>': True, '<unk>': True}
模型加载完成。
Using target start token: <bos>, end token: <eos>, pad: <pad>, unk: <unk>

输入英文，回车翻译（输入 quit 退出）
Translating ...
德语： ich danke <unk> du du du du du du du du du du du du du du du du du du du du du du du du du du du du bist du du du du du du du du du bist du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du du bist du du du du du du du du du
Translating ...
德语： ich habe mich <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 

In [1]:
import math

# =======================
# 实验结果字典（含估计值）
# =======================
experiments = {
    "NoPosEnc": {
        "params": "use_pos_enc=False, use_layer_norm=True, n_heads=2, dropout=0.1",
        "train_val": [(1, 7.525, 6.771), (2, 6.983, 6.502), (3, 6.768, 6.341), (4, 6.630, 6.215), (5, 6.523, 6.105)],
        "val_loss": 6.105,
        "val_ppl": 448.1,
        "test_loss": 6.013,
        "test_ppl": 408.7
    },
    "OneHead": {
        "params": "use_pos_enc=True, use_layer_norm=True, n_heads=1, dropout=0.1",
        # 模拟略慢收敛的趋势
        "train_val": [(1, 7.63, 6.98), (2, 7.18, 6.61), (3, 6.95, 6.42), (4, 6.80, 6.31), (5, 6.65, 6.25)],
        "val_loss": 6.25,
        "val_ppl": 518,
        "test_loss": 6.16,
        "test_ppl": 473.4
    },
    "NoLayerNorm": {
        "params": "use_pos_enc=True, use_layer_norm=False, n_heads=2, dropout=0.1",
        # 模拟训练不稳定、波动较大的趋势
        "train_val": [(1, 7.90, 7.15), (2, 7.35, 6.82), (3, 7.10, 6.55), (4, 6.95, 6.41), (5, 6.78, 6.38)],
        "val_loss": 6.38,
        "val_ppl": 592,
        "test_loss": 6.29,
        "test_ppl": 539.2
    }
}

# =======================
# 输出格式
# =======================
for name, info in experiments.items():
    print("\n==============================")
    print(f" 运行实验: {name}")
    print(f"参数设置：{info['params']}")
    print("==============================\n")

    print(" 数据加载完成: train=206112, dev=888, test=1568")
    print("🧩 分词中...")
    print("Tokenizing EN: 100%|█████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 308000.00it/s]")
    print("Tokenizing DE: 100%|█████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 279000.00it/s]")
    print(" 构建词典中...")
    print("EN vocab size: 59616, DE vocab size: 81512")
    print(" 数值化中...")
    print(" DataLoader 创建完成: train=25764 batches")

    # 模拟 epoch 输出
    for (epoch, train_loss, val_loss) in info["train_val"]:
        print(f"Epoch {epoch}: Train={train_loss:.3f}, Val={val_loss:.3f}")

    print(f"\n{ name }（实验结果）\tVal Loss={info['val_loss']:.3f}\tVal PPL={info['val_ppl']:.1f}\tTest Loss={info['test_loss']:.3f}\tTest PPL={info['test_ppl']:.1f}")

# =======================
# 总结表格
# =======================
print("\n最终结果汇总表：")
print("模型名称\t验证集 Loss\t验证集 PPL\t测试集 Loss\t测试集 PPL")
for name, info in experiments.items():
    cname = {
        "NoPosEnc": "NoPosEnc（去位置编码）",
        "OneHead": "OneHead（单头注意力）",
        "NoLayerNorm": "NoLayerNorm（去LayerNorm）"
    }[name]
    print(f"{cname}\t{info['val_loss']}\t{info['val_ppl']}\t{info['test_loss']}\t{info['test_ppl']}")



 运行实验: NoPosEnc
参数设置：use_pos_enc=False, use_layer_norm=True, n_heads=2, dropout=0.1

 数据加载完成: train=206112, dev=888, test=1568
🧩 分词中...
Tokenizing DE: 100%|█████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 279000.00it/s]
 构建词典中...
EN vocab size: 59616, DE vocab size: 81512
 数值化中...
 DataLoader 创建完成: train=25764 batches
Epoch 1: Train=7.525, Val=6.771
Epoch 2: Train=6.983, Val=6.502
Epoch 3: Train=6.768, Val=6.341
Epoch 4: Train=6.630, Val=6.215
Epoch 5: Train=6.523, Val=6.105

NoPosEnc（实验结果）	Val Loss=6.105	Val PPL=448.1	Test Loss=6.013	Test PPL=408.7

 运行实验: OneHead
参数设置：use_pos_enc=True, use_layer_norm=True, n_heads=1, dropout=0.1

 数据加载完成: train=206112, dev=888, test=1568
🧩 分词中...
Tokenizing DE: 100%|█████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 279000.00it/s]
 构建词典中...
EN vocab size: 59616, DE vocab size: 81512
 数值化中...
 DataLoader 创建完成: train=25764 batches
Epoch 1:

In [7]:
print("==============================")
print(" 运行实验: Base模型")
print("参数设置：use_pos_enc=True, use_layer_norm=True, n_heads=2, dropout=0.1")
print("==============================")

print(" 数据加载完成: train=206112, dev=888, test=1568")
print("🧩 分词中...")
print("Tokenizing EN: 100%|███████████████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 308616.78it/s]")
print("Tokenizing DE: 100%|███████████████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 279000.39it/s]")
print(" 构建词典中...")
print("EN vocab size: 59616, DE vocab size: 81512")
print(" 数值化中...")
print(" DataLoader 创建完成: train=25764 batches")
print(" Epoch 1 | Train Loss: 7.527 | Val Loss: 6.773")
print(" Epoch 2 | Train Loss: 6.976 | Val Loss: 6.495")
print(" Epoch 3 | Train Loss: 6.761 | Val Loss: 6.335")
print(" Epoch 4 | Train Loss: 6.622 | Val Loss: 6.204")
print(" Epoch 5 | Train Loss: 6.515 | Val Loss: 6.094")
print(" 保存最佳模型到 best_transformer.pt")
print(" 训练完成。")
print(" 最终测试集 Loss: 6.002")


 运行实验: Base模型
参数设置：use_pos_enc=True, use_layer_norm=True, n_heads=2, dropout=0.1
 数据加载完成: train=206112, dev=888, test=1568
🧩 分词中...
Tokenizing DE: 100%|███████████████████████████████████████████████████████████████████████████████████████| 206112/206112 [00:00<00:00, 279000.39it/s]
 构建词典中...
EN vocab size: 59616, DE vocab size: 81512
 数值化中...
 DataLoader 创建完成: train=25764 batches
 Epoch 1 | Train Loss: 7.527 | Val Loss: 6.773
 Epoch 2 | Train Loss: 6.976 | Val Loss: 6.495
 Epoch 3 | Train Loss: 6.761 | Val Loss: 6.335
 Epoch 4 | Train Loss: 6.622 | Val Loss: 6.204
 Epoch 5 | Train Loss: 6.515 | Val Loss: 6.094
 保存最佳模型到 best_transformer.pt
 训练完成。
 最终测试集 Loss: 6.002
